In [142]:
import numpy as np
import pandas as pd
import zipfile
import csv
import re
import tensorflow as tf
import pickle
from pycontractions import Contractions

In [ ]:
import gzip
import shutil
with gzip.open('obj/GoogleNews-vectors-negative300.bin.gz', 'rb') as f_in:
    with open('obj/GoogleNews-vectors-negative300.bin', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [143]:
pd.set_option('display.max_colwidth', -1)
df = pd.read_csv('data/olid-training-v1.0.tsv', sep='\t')
df.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans what their take on this is.,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of shit to a volcano. 😂""",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illegals to move into red states,NOT,NaN,NaN


In [144]:
X = df.drop(columns=['subtask_a', 'subtask_b','subtask_c'])
y = df.drop(columns=['id', 'tweet'])
X = df.filter(['id', 'tweet'])
X.head()

,id,tweet
0,86426,@USER She should ask a few native Americans what their take on this is.
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL
2,16820,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT
3,62688,"@USER Someone should'veTaken"" this piece of shit to a volcano. 😂"""
4,43605,@USER @USER Obama wanted liberals &amp; illegals to move into red states


In [145]:
def load_obj(name):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [146]:
contraction_map = load_obj("contraction_map")
print(contraction_map)

{"ain't": 'is not', "aren't": 'are not', "can't": 'cannot', "can't've": 'cannot have', "'cause": 'because', "could've": 'could have', "couldn't": 'could not', "couldn't've": 'could not have', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hadn't've": 'had not have', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'd've": 'he would have', "he'll": 'he will', "he'll've": 'he he will have', "he's": 'he is', "how'd": 'how did', "how'd'y": 'how do you', "how'll": 'how will', "how's": 'how is', "I'd": 'I would', "I'd've": 'I would have', "I'll": 'I will', "I'll've": 'I will have', "I'm": 'I am', "I've": 'I have', "i'd": 'i would', "i'd've": 'i would have', "i'll": 'i will', "i'll've": 'i will have', "i'm": 'i am', "i've": 'i have', "isn't": 'is not', "it'd": 'it would', "it'd've": 'it would have', "it'll": 'it will', "it'll've": 'it will have', "it's": 'it is', "let's": 'let us', "ma'am": 'madam', "mayn't": 'may not', "might've": 'migh

In [150]:
# alternatively, check out pycontractions
cont = Contractions('GoogleNews-vectors-negative300.bin')
cont.load_models()

Word2Vec model not found at GoogleNews-vectors-negative300.bin


SystemExit: 1

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [148]:
X['tweet'] = X['tweet'].apply(lambda x: cont.expand_texts(x, precise=True))

In [149]:
X.head(10)

,id,tweet
0,86426,<generator object Contractions.expand_texts at 0x139728518>
1,90194,<generator object Contractions.expand_texts at 0x139728570>
2,16820,<generator object Contractions.expand_texts at 0x1397285c8>
3,62688,<generator object Contractions.expand_texts at 0x139728620>
4,43605,<generator object Contractions.expand_texts at 0x139728678>
5,97670,<generator object Contractions.expand_texts at 0x1397286d0>
6,77444,<generator object Contractions.expand_texts at 0x139728728>
7,52415,<generator object Contractions.expand_texts at 0x139728780>
8,45157,<generator object Contractions.expand_texts at 0x1397287d8>
9,13384,<generator object Contractions.expand_texts at 0x139728830>


In [ ]:
def expand_contractions(sentence, contraction_mapping):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_sentence = contractions_pattern.sub(expand_match, sentence)
    return expanded_sentence



In [139]:
X['tweet'] = X['tweet'].apply(lambda x: ' '.join(re.sub("@USER"," ",x).split()))
X['tweet'] = X['tweet'].apply(lambda x: expand_contractions(x, contraction_map))

In [140]:
X.head(10)

,id,tweet
0,86426,She should ask a few native Americans what their take on this is.
1,90194,Go home you’re drunk!!! #MAGA #Trump2020 👊🇺🇸👊 URL
2,16820,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT
3,62688,"Someone should haveTaken"" this piece of shit to a volcano. 😂"""
4,43605,Obama wanted liberals &amp; illegals to move into red states
5,97670,Liberals are all Kookoo !!!
6,77444,Oh noes! Tough shit.
7,52415,was literally just talking about this lol all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism
8,45157,Buy more icecream!!!
9,13384,Canada doesn’t need another CUCK! We already have enough #LooneyLeft #Liberals f**king up our great country! #Qproofs #TrudeauMustGo


In [123]:
X['tweet'] = X['tweet'].apply(lambda x: x.lower())
X['tweet'] = X['tweet'].apply(lambda x: ' '.join(re.sub("(@[A-Za-z0-9]+)|#|url|&amp"," ",x).split()))
X['tweet'] = X['tweet'].apply(lambda x: ' '.join(re.sub("[?.,!;]"," [SEP]", x).split()))
X['tweet'] = X['tweet'].apply(lambda x: ' '.join(re.sub("([^0-9A-Za-z \[\]\t])|(\w+:\/\/\S+)"," ",x).split()))
X['tweet'] = X['tweet'].apply(lambda x: '[CLS] {} [SEP]'.format(x))


In [124]:
X.head()




,id,tweet
0,86426,[CLS] she should ask a few native americans what their take on this is [SEP] [SEP]
1,90194,[CLS] go home you re drunk [SEP] [SEP] [SEP] maga trump2020 [SEP]
2,16820,[CLS] amazon is investigating chinese employees who are selling internal data to third party sellers looking for an edge in the competitive marketplace [SEP] amazon maga kag china tcot [SEP]
3,62688,[CLS] someone should havetaken this piece of shit to a volcano [SEP] [SEP]
4,43605,[CLS] obama wanted liberals [SEP] illegals to move into red states [SEP]


In [125]:
X.head(10)

,id,tweet
0,86426,[CLS] she should ask a few native americans what their take on this is [SEP] [SEP]
1,90194,[CLS] go home you re drunk [SEP] [SEP] [SEP] maga trump2020 [SEP]
2,16820,[CLS] amazon is investigating chinese employees who are selling internal data to third party sellers looking for an edge in the competitive marketplace [SEP] amazon maga kag china tcot [SEP]
3,62688,[CLS] someone should havetaken this piece of shit to a volcano [SEP] [SEP]
4,43605,[CLS] obama wanted liberals [SEP] illegals to move into red states [SEP]
5,97670,[CLS] liberals are all kookoo [SEP] [SEP] [SEP] [SEP]
6,77444,[CLS] oh noes [SEP] tough shit [SEP] [SEP]
7,52415,[CLS] was literally just talking about this lol all mass shootings like that have been set ups [SEP] it s propaganda used to divide us on major issues like gun control and terrorism [SEP]
8,45157,[CLS] buy more icecream [SEP] [SEP] [SEP] [SEP]
9,13384,[CLS] canada doesn t need another cuck [SEP] we already have enough looneyleft liberals f king up our great country [SEP] qproofs trudeaumustgo [SEP]
